# **Lab 1: Matrix factorization**
**Gustav Grevsten**

# **Abstract**

The purpose of this lab is to implement and test algorithms for calculating a sparse matrix-vector product, QR factorization of matrices that are real, quadratic and invertible, and for directly solving for vectors $x=A^{-1}b$ for a given $b$ and $A$. In the end, we conclude that the algorithms implemented yield results that were expected.



# **Set up environment**

In [1]:
# Load neccessary modules.
import numpy as np

# **Introduction**

Linear operators play an important role in many scientific computations, as they provide a powerful mathematical tool for modeling real-world phenomena. To this end, it is often useful to be able to effectivize computations involving them and to use various techniques for comptuting difficult problems.

The compressed row storage (CRS) format is a technique for storing sparse matrices, which are matrices with mostly zeroes in their entries. This is useful in many applications, such as numerical simulations, optimization, and data analysis, where large sparse matrices arise naturally. The sparse structure of the matrices means that the CRS format can be used for computing matrix-vector products with reduced computational cost and memory requirements.

QR factorization is a matrix decomposition technique that factorizes a matrix into an orthogonal matrix and an upper-triangular matrix. The QR factorization of a matrix has many useful properties, such as numerical stability and orthogonality, which make it a valuable tool in many applications. QR factorization can also be utilized in order to calculate the inverses of matrices with a relatively simple algorithm.

# **Method**

The algorithms we will employ in this lab are those for calculating a sparse matrix vector product, QR-factorization of real, quadratic and invertible matrices, and using the QR-factorization of a matrix $A$ to directly solve the problem $x=A^{-1}b$.

First, we implement the algorithm for the sparse matrix-vector product using the CRS format. Here, we avoid calculating the products of the zero elements in the matrix, saving computational time and making the input size smaller to store.

In [2]:
def sparse_matrix_vector_product(val, col_idx, row_ptr, x):
  n = len(x)
  b = np.zeros(n)
  for i in range(n):
    for j in range(row_ptr[i]-1, row_ptr[i+1]-1):
      b[i] += val[j]*x[col_idx[j]-1]
  return b

Next, we implement the algorithm for calculating QR factorization of a full rank matrix $A$. This algorithm uses a modified version of the Gram–Schmidt process in order to construct an orthogonal set of vectors for the column space of $Q$, while calculating the corresponding upper-triangular matrix $R$ in parallel. These fulfill the criteria $QR = A$.

In [3]:
def QR_factorization(A):
  n = len(A)
  R = np.zeros((n, n))
  Q = np.zeros((n, n))
  v = A[:,0]
  for i in range(n):
    Q[i,0] = v[i]/np.linalg.norm(v)
  R[0,0] = np.linalg.norm(v)
  for j in range(1, n):
    v = A[:,j].copy()
    v_copy = v.copy()
    for i in range(j):
      qi = Q[:,i]
      R[i,j] = np.dot(qi, v_copy)
      v = np.subtract(v, np.multiply(R[i,j], qi))
    R[j,j] = np.linalg.norm(v)
    for index in range(n):
      Q[index,j] = v[index]/R[j,j]
  return Q, R

Finally, we implement the algorithm for computing the solution $x=A^{-1}b$ for a matrix $A$ of full rank. To this end, we can make use of QR factorization, since $A = QR \implies A^{-1} = (QR)^{-1} = R^{-1}Q^{T}$. The transpose of $Q$ is easy to compute, and the upper-triangular nature of $R$ means that we can easily find the solution to $x=R^{-1}(Q^{T}b)$ using back-substitution. We can solve for $x$ by solving the equations for $x_n$, then $x_{n-1}$ and so on until we find $x_1$.

In [4]:
def direct_solve(A, b):
  Q, R = QR_factorization(A)
  n = len(b)
  x = np.matmul(np.transpose(Q), b)
  x_copy = x.copy()
  x[n-1] = x[n-1]/R[n-1,n-1]
  for i in range(n-2, -1, -1):
    sum = 0
    for j in range(i+1, n):
      sum += R[i,j]*x[j]
    x[i] = (x[i] - sum)/R[i,i]
  return x

# **Results**

We test the algorithms presented in the methods section below. We start by showing that we can calculate the sparse matrix vector product between the arbitrary matrix

$
A = 
\left[
  \begin{array}{ccccc}
    3 & 1 & 0 & 0 & 4\\
    0 & 1 & 5 & 0 & 0 \\
    0 & 9 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 \\
    0 & 2 & 6 & 0 & 5 \\
  \end{array}
\right]
$

and the vector

$
x = 
\left[
  \begin{array}{c}
    1\\
    2\\
    3\\
    4\\
    5\\
  \end{array}
\right].
$

Using the CRS format, we express $A$ as

$val = [3, 1, 4, 1, 5, 9, 2, 6, 5]$, 
$col\_idx = [1, 2, 5, 2, 3, 2, 2, 3, 5]$, and $row\_ptr = [1, 4, 6, 7, 7, 10].$

We then compare this to the result yielded by the dense matrix-vector product.

In [5]:
val = [3, 1, 4, 1, 5, 9, 2, 6, 5] 
col_idx = [1, 2, 5, 2, 3, 2, 2, 3, 5] 
row_ptr = [1, 4, 6, 7, 7, 10]

A = np.array([
    [3, 1, 0, 0, 4], 
    [0, 1, 5, 0, 0], 
    [0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 2, 6, 0, 5],
    ])

x = [1,2,3,4,5]

print("Sparse matrix-vector product: " + str(sparse_matrix_vector_product(val, col_idx, row_ptr, x)))

print("Dense matrix-vector product: " + str(np.matmul(A,x)))

Sparse matrix-vector product: [25. 17. 18.  0. 47.]
Dense matrix-vector product: [25 17 18  0 47]


Which shows that the sparse matrix-vector product yields the same result as the dense matrix-vector product.

Next, we test the algorithm for QR factorization by applying it to the arbitrary matrix of full rank
$
A = 
\left[
  \begin{array}{ccc}
    3 & 1 & 4\\
    1 & 5 & 9\\
    2 & 6 & 5\\
  \end{array}
\right].
$

For the QR factorization, it should hold that $|| Q^TQ-I ||_F = 0$ and $|| QR-A ||_F = 0$, since $Q$ is orthogonal and $QR=A$

In [6]:
A = np.array([
    [3, 1, 4], 
    [1, 5, 9], 
    [2, 6, 5]
    ])

I = np.array([
    [1, 0, 0], 
    [0, 1, 0], 
    [0, 0, 1]
    ])

Q, R = QR_factorization(A)

A_QR = np.matmul(Q,R)

I_QR = np.matmul(np.transpose(Q),Q)

print("Matrix R: ")

print(R)

print("")

print("||QR - A||_F = " + str(np.linalg.norm(A_QR - A)))

print("")

print("||Q^T Q - I||_F = " + str(np.linalg.norm(I_QR - I)))

Matrix R: 
[[3.74165739 5.34522484 8.2850985 ]
 [0.         5.78174467 6.00411946]
 [0.         0.         4.16025147]]

||QR - A||_F = 1.9984014443252818e-15

||Q^T Q - I||_F = 3.906404417713938e-16


As we can clearly see, the matrix R is upper triangular, and the Frobenius norms are both very close to zero, as expected.

Finally, we test the direct solver for the matrix

$
A = 
\left[
  \begin{array}{ccc}
    3 & 1 & 4\\
    1 & 5 & 9\\
    2 & 6 & 5\\
  \end{array}
\right]
$

multiplied by the vector

$
x = 
\left[
  \begin{array}{c}
    1\\
    2\\
    3\\
  \end{array}
\right]
$

which should yield the solution

$
Ax = b =
\left[
  \begin{array}{c}
    17\\
    38\\
    29\\
  \end{array}
\right]
$

In [8]:
A = np.array([
    [3, 1, 4], 
    [1, 5, 9], 
    [2, 6, 5]
    ])

x = [1, 2, 3]

b = np.matmul(A,x)

x_calculated = direct_solve(A, b)

Ax = np.matmul(A,x_calculated)

print("||Ax - b|| = " + str(np.linalg.norm(Ax - b)))

print("||x - y|| = " + str(np.linalg.norm(x - x_calculated)))



||Ax - b|| = 1.0658141036401503e-14
||x - y|| = 2.9790409838967277e-15


As we can see, the direct solver managed to find a very close approximation of the vector x, as expected.

# **Discussion**

As expected, we were able to find good approximate solutions to the various problems. We used the CRS format for storing a matrix and calculating a vector-matrix product, and we used an algorithm for finding QR factorizations, which we then used in order to calculate an inverse of a matrix.

Observe that the CRS format is only more efficient for matrices with many zero elements, as densely populated matrices will require more space if stored in this way.

Also, the QR factorization algorithm will not always produce orthogonal matrices $Q$ if the matrix $A$ is singular. Below is an example for the singular matrix

$
A = 
\left[
  \begin{array}{ccc}
    1 & 2 & 3\\
    4 & 5 & 6\\
    7 & 8 & 9\\
  \end{array}
\right]
$

which yields a non-orthogonal matrix $Q$.

In [12]:
A = np.array([
    [1, 2, 3], 
    [4, 5, 6], 
    [7, 8, 9]
    ])

I = np.array([
    [1, 0, 0], 
    [0, 1, 0], 
    [0, 0, 1]
    ])

Q, R = QR_factorization(A)

I_QR = np.matmul(np.transpose(Q),Q)

print("")

print("||Q^T Q - I||_F = " + str(np.linalg.norm(I_QR - I)))


||Q^T Q - I||_F = 1.4142135623730951
